In [57]:
import pickle
import pandas as pd

In [58]:
train = open("train_large",'rb')
train_data = pickle.load(train)
train_cleaned = [["".join(value[0]), value[1]] for key, value in train_data.items()]

In [59]:
RT_removed = []
cleansed = []
for title in train_cleaned:
    t = title[0].split(" ")
    if "RT" in title[0]:
        RT_removed.append([" ".join(t[2:]), title[1]])
    else:
        RT_removed.append(title)

for title in RT_removed:
    t = title[0].split(" ")
    if "via" in title[0]:
        for i in t:
            if "via" in i and not i.istitle():
                indx = t.index(i)
                cleansed.append([" ".join(t[:indx]), title[1]])
    else:
        cleansed.append(title)

In [69]:
data = pd.DataFrame(cleansed, columns=["Title", "Clickbait"])
data.to_html('data.html')

In [68]:
%matplotlib inline
from matplotlib import pyplot as plt
lengths_cb = []
lengths_ncb = []
for data in cleansed:
    if data[1] is 1:
        lengths_cb.append(len(data[0].split(" ")))
    else:
        lengths_ncb.append(len(data[0].split(" ")))
print(pd.Series(lengths_cb).mean())
print(pd.Series(lengths_ncb).mean())

10.060294117647059
12.028987470369117


In [43]:
dfx = data["Title"]
dfy = data["Clickbait"]

In [44]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
count_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, stop_words='english')

In [46]:
count_vec = count_vectorizer.fit_transform(dfx)
tdidf_vec = tfidf_vectorizer.fit_transform(dfx)

In [47]:
from sklearn.neural_network import MLPClassifier

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(count_vec, dfy, test_size=0.3, random_state=1)

In [49]:
mlp = MLPClassifier(activation="relu", solver="sgd", hidden_layer_sizes=(7), random_state=1)

In [50]:
mlp.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=7, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [51]:
predict = mlp.predict(X_test)
real_val = y_test.values


counter = 0
for i in range(len(predict)):
    if predict[i] == real_val[i]:
        counter += 1
counter/len(predict)

0.7303523035230353

## 10-fold CV

In [52]:
from sklearn.model_selection import cross_val_score
import numpy as np

In [55]:
clf = MLPClassifier(activation="identity", solver="sgd", hidden_layer_sizes=(1,3), random_state=2)
cvs = cross_val_score(clf, count_vec, dfy, cv=10, scoring='accuracy')

/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/ana

In [56]:
cvs.mean()

0.7494265817317348

### TF-IDF

In [157]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tdidf_vec, dfy, test_size=0.3, random_state=0)

In [158]:
mlp = MLPClassifier(activation="relu", solver="sgd", hidden_layer_sizes=(10,10), random_state=1, max_iter=500)

In [159]:
mlp.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [160]:
predict = mlp.predict(X_test)
real_val = y_test.values


counter = 0
for i in range(len(predict)):
    if predict[i] == real_val[i]:
        counter += 1
counter/len(predict)

0.6856368563685636